# VacationPy
---

## Used the starter code as a template for the challenge.

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,constantia,44.1833,28.6500,74.08,50,0,7.67,RO,1720471446
1,1,tiksi,71.6872,128.8694,50.95,59,0,4.27,RU,1720471502
2,2,port-aux-francais,-49.3500,70.2167,36.21,86,10,32.55,TF,1720471503
3,3,tsiombe,-25.3000,45.4833,61.77,74,0,4.88,MG,1720471504
4,4,mugia,43.1041,-9.2179,64.90,96,100,19.35,ES,1720471506


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:


# Configure the map plot
map_plot = city_data_df.hvplot.points(
	'Lng', 'Lat',
	geo=True,
	tiles='OSM',
	size='Humidity',
	color= 'City',
	cmap= 'Category20',
	frame_width=700,
	frame_height=500,
)

# Display the map
map_plot

a:\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
clean_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 10)].dropna()

# Drop any rows with null values
clean_city_data_df = clean_city_data_df.dropna()

# Display sample data
clean_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,constantia,44.1833,28.6500,74.08,50,0,7.67,RO,1720471446
103,103,ariano irpino-martiri,41.1489,15.0833,73.38,62,0,1.79,IT,1720471619
124,124,sona,45.4388,10.8255,78.35,72,0,5.75,IT,1720471643
152,152,baile herculane,44.8797,22.4125,74.64,54,0,5.82,RO,1720471674
153,153,fort st. john,56.2499,-120.8529,78.69,41,0,3.44,CA,1720471451


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,constantia,RO,44.1833,28.6500,50,
103,ariano irpino-martiri,IT,41.1489,15.0833,62,
124,sona,IT,45.4388,10.8255,72,
152,baile herculane,RO,44.8797,22.4125,54,
153,fort st. john,CA,56.2499,-120.8529,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "type": "hotel",
    "key": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{radius}@{lat},{row["Lng"]}'
    params["bias"] = f'proximity:{lat},{row["Lng"]}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    request = requests.get(base_url, params=params).json()

    # Convert the API response to JSON format
    name_address = request

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
constantia - nearest hotel: No hotel found
ariano irpino-martiri - nearest hotel: No hotel found
sona - nearest hotel: No hotel found
baile herculane - nearest hotel: No hotel found
fort st. john - nearest hotel: No hotel found
santa eularia des riu - nearest hotel: No hotel found
saint-francois - nearest hotel: No hotel found
la junta - nearest hotel: No hotel found
sant lluis - nearest hotel: No hotel found
bani walid - nearest hotel: No hotel found
zakynthos - nearest hotel: No hotel found
saint-denis - nearest hotel: No hotel found
pozoblanco - nearest hotel: No hotel found
tiszafoldvar - nearest hotel: No hotel found
siklos - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,constantia,RO,44.1833,28.6500,50,No hotel found
103,ariano irpino-martiri,IT,41.1489,15.0833,62,No hotel found
124,sona,IT,45.4388,10.8255,72,No hotel found
152,baile herculane,RO,44.8797,22.4125,54,No hotel found
153,fort st. john,CA,56.2499,-120.8529,41,No hotel found
173,santa eularia des riu,ES,38.9846,1.5341,85,No hotel found
176,saint-francois,FR,46.4154,3.9054,68,No hotel found
239,la junta,US,37.9850,-103.5438,44,No hotel found
255,sant lluis,ES,39.8526,4.2594,78,No hotel found
287,bani walid,LY,31.7566,13.9942,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
	'Lng', 'Lat',
	geo=True,
	tiles='OSM',
	size='Humidity',
	color='Hotel Name',
	cmap='Category20',
	frame_width=700,
	frame_height=500,
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity)